# Environment Set-up & Verification

This Jupyter Notebook is designed as a preliminary step for verifying proper environmental set-up using either the provided environment.yml file or another manually designed environment. 

** To be added to project Notebook eventually.

In [11]:
import importlib
import sys

packages = [
    "python",
    "numpy",
    "scipy",
    "pandas",
    "matplotlib",
    "sklearn",
    "hyperspy",
    "ipywidgets",
    "seaborn",
    "tqdm"
]

print("Environment check")
print("=" * 50)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")
print("=" * 50)

for pkg in packages: 
    try:
        if pkg == "python":
            continue
        module = importlib.import_module(pkg)
        version = getattr(module, "__version__", "No__version__attribute")
        print(f"{pkg:15s} : {version}")
    except ImportError:
        print(f"{pkg:15s} : NOT INSTALLED")

# HyperSpy is the primary library for this project. Please explicitly confirm it's loading.

import hyperspy.api as hs
print("\nHyperSpy API successfully imported.")

Environment check
Python executable: C:\Users\elise\miniforge3\envs\zincsoap-ml\python.exe
Python version: 3.10.19
numpy           : 1.26.4
scipy           : 1.15.2
pandas          : 2.3.3
matplotlib      : 3.10.8
sklearn         : 1.7.2
hyperspy        : 2.4.0
ipywidgets      : 8.1.8
seaborn         : 0.13.2
tqdm            : 4.67.3

HyperSpy API successfully imported.
